In [ ]:
import pandas

import automol
import chemkin_io
import helpers
import ioformat

In [ ]:
# 1. read in the species lists
C67_SPC_PATH = 'mechanisms/01_raw_nuig-c6-7.csv'
PYR_SPC_PATH = 'mechanisms/01_raw_nuig-pyro.csv'

C67_SPC_DF = pandas.read_csv(C67_SPC_PATH, quotechar="'")
PYR_SPC_DF = pandas.read_csv(PYR_SPC_PATH, quotechar="'")

In [ ]:
# 2. read in the reaction lists
C67_RXN_PATH = 'mechanisms/01_raw_nuig-c6-7.txt'
PYR_RXN_PATH = 'mechanisms/01_raw_nuig-pyro.txt'

C67_MECH_STR = ioformat.pathtools.read_file('.', C67_RXN_PATH)
PYR_MECH_STR = ioformat.pathtools.read_file('.', PYR_RXN_PATH)

C67_RXN_DCT = chemkin_io.parser.mechanism.reactions(C67_MECH_STR)
PYR_RXN_DCT = chemkin_io.parser.mechanism.reactions(PYR_MECH_STR)

In [ ]:
def hash_reaction_from_chi(rct_chis, prd_chis):
    """ create a hash for a reaction from its ChIs
    """
    rct_key = automol.inchi_key.first_hash(
        automol.chi.inchi_key(automol.chi.join(rct_chis)))
    prd_key = automol.inchi_key.first_hash(
        automol.chi.inchi_key(automol.chi.join(prd_chis)))
    return f'{rct_key}_{prd_key}'


def classify_reactions(rxn_dct, spc_df):
    """ classifies reactions
    """
    row_dcts = []
    for rxn, params in rxn_dct.items():
        name = chemkin_io.writer.format_rxn_name(rxn)
        print(f'Classifying {name}...')
        
        row_dct = {
            'name': name,
            'orig-name': name,
            'params': params,
            'succeeded': False,
            'class': None,
            'path': None,
        }

        # First, classify the reaction from InChIs
        rct_names, prd_names, third_body = rxn
        rct_chis = [helpers.lookup_value(spc_df, 'inchi', {'name': n}) for n in rct_names]
        prd_chis = [helpers.lookup_value(spc_df, 'inchi', {'name': n}) for n in prd_names]

        rxn_objs = []
        fail_message = ''
        try:
            rxn_objs = automol.reac.from_chis(rct_chis, prd_chis, stereo=False)
        except Exception as e:
            fail_message = f' - Failed with error {e}'

        if not rxn_objs:
            print(' - Failed')
            print(fail_message if fail_message else ' - Unclassified')

        for num, rxn_obj in enumerate(rxn_objs):
            name = chemkin_io.writer.format_rxn_name(
                (rct_names, prd_names, third_body))

            ts_chi = automol.reac.ts_amchi(rxn_obj)

            print(' - Succeeded')
            rxn_hash = hash_reaction_from_chi(rct_chis, prd_chis)
            rxn_obj_str = automol.reac.string(rxn_obj)
            rxn_obj_path = f'reactions/graphs/{rxn_hash}_{num}.txt'
            ioformat.pathtools.write_file(rxn_obj_str, '.', rxn_obj_path)
            row_dct.update({
                'name': name,
                'succeeded': True,
                'class': rxn_obj.class_,
                'path': rxn_obj_path,
                'ts_amchi': ts_chi,
            })

        row_dcts.append(row_dct)
        
    rxn_df = pandas.DataFrame.from_records(row_dcts)
    return rxn_df

In [ ]:
# 3. classify reactions, extract successes/failures, and write them to files
print("Classifying reactions for for NUIG-C6-7")
C67_RXN_DF = classify_reactions(C67_RXN_DCT, C67_SPC_DF)
SUCC_C67_RXN_PATH = 'reactions/03_reactions-succ_nuig-c6-7.csv'
FAIL_C67_RXN_PATH = 'reactions/03_reactions-fail_nuig-c6-7.csv'

SUCC_C67_RXN_DF = C67_RXN_DF[C67_RXN_DF['succeeded']]
FAIL_C67_RXN_DF = C67_RXN_DF[~C67_RXN_DF['succeeded']]

SUCC_C67_RXN_DF.to_csv(SUCC_C67_RXN_PATH, quotechar="'")
FAIL_C67_RXN_DF.to_csv(FAIL_C67_RXN_PATH, quotechar="'")

 - Succeeded
Classifying I3C6 + CH3 = I3C6-1 + CH4...
 - Succeeded
Classifying TC4H9O2 + CH3CO3 => TC4H9O + CH3CO2 + O2...
 - Failed
 - Unclassified
Classifying C4H8-1 + C3H5-A = C4H71-3 + C3H6...
 - Succeeded
Classifying IC5H12 + CH3O2 = CC5H11 + CH3O2H...
 - Succeeded
Classifying C7H13OH1Q2-5O2 = C7H124OH1Q2 + HO2...
 - Succeeded
Classifying I3C6OOH2-3O2 = I3C6Q23-6...
 - Succeeded
Classifying C6H11Q13-6 = C4H71-2,4OOH + C2H4...
 - Succeeded
Classifying IC6D4 + CH3O2 = IC6D4-1 + CH3O2H...
 - Succeeded
Classifying C7H121KET36 = C7H121KET36O + OH...
 - Succeeded
Classifying IC6-4 + CH3O2 = IC6-4O + CH3O...
 - Succeeded
Classifying C7H14OOH2-3O2 = C7H13Q23-6...
 - Succeeded
Classifying XC6 + CH3O = XC6-1 + CH3OH...
 - Succeeded
Classifying CVCCVCCOJC = CH3CHO + C4H5-N...
 - Succeeded
Classifying IC6-3OOH = IC6-3O + OH...
 - Succeeded
Classifying IC6D3 + HO2 = IC6D3-1 + H2O2...
 - Succeeded
Classifying IC6OH2-1J + O2 = IC6OH2-1O2...
 - Succeeded
Classifying SC4H9O2 + NC3H7 = SC4H9O + NC3

 - Succeeded
Classifying C6H12OOH2-3O2 = C6KET23 + OH...
 - Failed
 - Unclassified
Classifying C6H111-3 = C6H112-5...
 - Succeeded
Classifying C6H11Q15-6 = C6H11O12-6OOH + OH...
 - Succeeded
Classifying C7H14OOH3-1O2 = C7H13Q13-2...
 - Succeeded
Classifying XC6-1O2 + CH3O2 => XC6-1O + CH3O + O2...
 - Failed
 - Unclassified
Classifying IC6D1 + HO2 = IC6D1-3 + H2O2...
 - Succeeded
Classifying C6H13O2-2 = C6H12-1 + HO2...
 - Succeeded
Classifying C6H12OH-1O2-2 = C6H11OH1Q2-3...
 - Succeeded
Classifying C7H121OOH6-4O2 = C7H11D13-6OOH + HO2...
 - Succeeded
Classifying C6H5CH2OH = C6H5CH2 + OH...
 - Succeeded
Classifying IC6Q35-4 = IC6D3-5OOH + HO2...
 - Succeeded
Classifying C4H7CHO-2 + CH3O2 = C5D2Y1-1R + CH3O2H...
 - Succeeded
Classifying C6H111-2 = C6H111-5...
 - Succeeded
Classifying C6H11Q15-2 = C6H11O25-1OOH + OH...
 - Succeeded
Classifying C7KET31O = CH2O + C6Y2-1J...
 - Succeeded
Classifying C7H14OOH3-5O2 = C7KET35 + OH...
 - Succeeded
Classifying IC6O23-1OOH => CH3CH2CHCOCH3 + CH2O

 - Succeeded
Classifying C6H12-2 + O2CHO = C6H112-5 + HO2CHO...
 - Succeeded
Classifying IC6OOH1-4O2 = IC6Q14-1A...
 - Succeeded
Classifying I3C6OOH1-5O2 = I3C6Q15-2...
 - Succeeded
Classifying IC6OOH3-5O2 = IC6Q35-2...
 - Succeeded
Classifying IC6-3O = C2H5CHO + IC3H7...
 - Succeeded
Classifying C6H13OOH-2 + O2 = C6H13O2-2 + HO2...
 - Succeeded
Classifying XC6D1 + O2 = XC6D1-1A + HO2...
 - Succeeded
Classifying C7H131-4O2 = C7H121-3 + HO2...
 - Succeeded
Classifying IC6OOH1-3O2 = IC6Q13-4...
 - Succeeded
Classifying NC6H14 = CH3 + C5H11-1...
 - Succeeded
Classifying XC6O12-3OOH => CH3COCH2 + CH3COCH3 + OH...
 - Failed
 - Unclassified
Classifying C7KET12O = NC5H11CHO + HCO...
 - Succeeded
Classifying IC6Q15-1A = AC3H5OOH + C3H6OOH1-3...
 - Succeeded
Classifying C7H13Q24-3 = C7H13O23-4OOH + OH...
 - Succeeded
Classifying IC6OOH3-5 + O2 = IC6OOH3-5O2...
 - Succeeded
Classifying C6H12OOH3-2O2 = C6KET32 + OH...
 - Failed
 - Unclassified
Classifying NEC6OOH1-4O2 = NEC6Q14-3...
 - Succeeded


 - Succeeded
Classifying I3C6-4O2 + CH3O2 => I3C6-4O + CH3O + O2...
 - Failed
 - Unclassified
Classifying C6H12OOH2-3O = C3KET12 + NC3H7...
 - Succeeded
Classifying I3C6Q25-1 = I3C6O15-2OOH + OH...
 - Succeeded
Classifying C6H101OOH3-6 + O2 = C6H101OOH3-6O2...
 - Succeeded
Classifying IC6D1 + OH = IC6D1-5 + H2O...
 - Succeeded
Classifying SC4H9CHO + CH3 = I3C6-2O...
 - Succeeded
Classifying I3C6OH2-3O2 => CH3CHO + C2H5COCH3 + OH...
 - Failed
 - Unclassified
Classifying IC6D4 + O2 = IC6D4-3 + HO2...
 - Succeeded
Classifying C7H121-4 + O2 = CVCCJCVCCC + HO2...
 - Succeeded
Classifying XC6D13 + H = C3H5-T + C3H6...
 - Succeeded
Classifying IC6KET4-3 = IC6KET4-3O + OH...
 - Succeeded
Classifying NC3H7CHO + C2H5O = C3H6CHO-2 + C2H5OH...
 - Succeeded
Classifying C6H10D24 = P13DE1J + CH3...
 - Succeeded
Classifying I3C6D2-5 + HO2 = I3C6D2-5O + OH...
 - Succeeded
Classifying I3C6OOH1-3 + O2 = I3C6OOH1-3O2...
 - Succeeded
Classifying C7H14OOH1-3 + O2 = C7H14OOH1-3O2...
 - Succeeded
Classifying 

 - Succeeded
Classifying C6H12OOH2-6O2 = C6H11Q15-2...
 - Succeeded
Classifying CC5H9-B + C2H5O2 = CC5H9O-B + C2H5O...
 - Succeeded
Classifying C7H14OOH3-2O2 = C7H13Q23-1...
 - Succeeded
Classifying C4H71-3 + C2H3 = C4H6 + C2H4...
 - Succeeded
Classifying FULVENE = C6H6...
 - Failed
 - Unclassified
Classifying C6H101OOH6-5O2 = NC6D1KET65 + OH...
 - Failed
 - Unclassified
Classifying XC6OOH1-1AO2 = XC6Q11A-4...
 - Succeeded
Classifying IC6D4-3 + HO2 = IC6D4-3O + OH...
 - Succeeded
Classifying C6H11OH1Q2-5O2 = C6H105OH1Q2 + HO2...
 - Succeeded
Classifying IC6D1-5 + O2 = IC6D14 + HO2...
 - Succeeded
Classifying C7H13Q34-5 = C7H13O34-5OOH + OH...
 - Succeeded
Classifying IC6D1-4 = IC6D13 + H...
 - Succeeded
Classifying XC6D1 + CH3 = XC6D1-1A + CH4...
 - Succeeded
Classifying C7H121OOH7-4O2 = C7H11D13-7OOH + HO2...
 - Succeeded
Classifying C7H14OOH2-4O2 = C7H13-3D2OOH + HO2...
 - Succeeded
Classifying I3C6OOH1-6O2 = I3C6KET1-6 + OH...
 - Succeeded
Classifying C6H101OOH4-6O2 = C6H9-AOOH + HO

...

 - Succeeded
Classifying IC6Q24-1 = IC6D1-4OOH + HO2...
 - Succeeded
Classifying HOC6H4CH3 + H = HOC6H4CH2 + H2...
 - Succeeded
Classifying I3C6Q13-2 = I3C6D1-3OOH + HO2...
 - Succeeded
Classifying C7H15-1 = C7H15-2...
 - Succeeded
Classifying XC6OOH1-3 = C5H9B-AOOH + CH3...
 - Failed
 - Unclassified
Classifying C2H5COC2H5 + CH3O2 = CH2CH2COC2H5 + CH3O2H...
 - Succeeded
Classifying NC3H7CHO + C2H3 = C3H6CHO-2 + C2H4...
 - Succeeded
Classifying I3C6D2 + O2CHO = I3C6D3-2 + HO2CHO...
 - Succeeded
Classifying C7H121KET67O => C4H71-4 + CH2CO + CH2O...
 - Failed
 - Unclassified
Classifying IC6D1 + O2 = IC6D1-4 + HO2...
 - Succeeded
Classifying C6H12OOH3-2O2 = C6H11Q23-5...
 - Succeeded
Classifying XC6D1 + OH = XC6D1-4 + H2O...
 - Succeeded
Classifying I3C6OOH3-4 + O2 = I3C6OOH3-4O2...
 - Succeeded
Classifying C7H15-3O2 = C7H14OOH3-2...
 - Succeeded
Classifying IC6Q13-2 = IC6O23-1OOH + OH...
 - Succeeded
Classifying I3C6O15-4OOH => C5H9O2-5 + CH2O + OH...
 - Failed
 - Unclassified
Classifying

 - Succeeded
Classifying C7H121-4 + HO2 = CVCCJCVCCC + H2O2...
 - Succeeded
Classifying C6H11Q13-6 = C6H11O14-6OOH + OH...
 - Succeeded
Classifying IC3H7O2 + NC3H7 = IC3H7O + NC3H7O...
 - Succeeded
Classifying IC6OOH4-1O2 = IC6Q14-1A...
 - Succeeded
Classifying C7H13Q35-4 = C7H13-3D5OOH + HO2...
 - Succeeded
Classifying BC5H11 + CH3 = NEC6...
 - Succeeded
Classifying IC4H9O2 + C2H5OH = IC4H9O2H + PC2H4OH...
 - Succeeded
Classifying BZCOOH = C6H5CHOH + OH...
 - Failed
 - Unclassified
Classifying NC5H11CHO + HO2 = NC5H10CHO-5 + H2O2...
 - Succeeded
Classifying C7H15-3O2 = C7H14OOH3-6...
 - Succeeded
Classifying I3C6OOH1-3O2 = I3C6Q13-6...
 - Succeeded
Classifying IC6OOH5-3 = IC6O3-5 + OH...
 - Succeeded
Classifying CH2CO + PC4H9 = C6Y2-1J...
 - Succeeded
Classifying C6H5OO = C6H5O + O...
 - Succeeded
Classifying SC4H9CHO + CH3O2 = SC4H9CO + CH3O2H...
 - Succeeded
Classifying IC6D3-1 = C5H81-3 + CH3...
 - Succeeded
Classifying C7H14-2 + H = C7H15-2...
 - Succeeded
Classifying C6H12OOH1-4O

 - Succeeded
Classifying C6H12-2 + OH = C6H12OH-2J3...
 - Succeeded
Classifying C6H111O2-6 = C6H101-5 + HO2...
 - Succeeded
Classifying I3C6D2 + OH = C2H5COCH3 + C2H5...
 - Failed
 - Unclassified
Classifying IC6O45-2OOH => CH3COCH3 + C3H5O + OH...
 - Failed
 - Unclassified
Classifying I3C6Q12-5 = I3C6O15-2OOH + OH...
 - Succeeded
Classifying XC6Q11A-2 = XC6O12-1AOOH + OH...
 - Succeeded
Classifying C7KET42O + O2 = C7Y24 + HO2...
 - Succeeded
Classifying CC5H10 + CH3 = CC5H9-B + CH4...
 - Succeeded
Classifying C7H14OOH3-7 + O2 = C7H14OOH3-7O2...
 - Succeeded
Classifying NEC6-3O2 + H2O2 = NEC6-3OOH + HO2...
 - Succeeded
Classifying C5H9OH1-1 + CH3O2 => C4H7CHO-2 + H + CH3O2H...
 - Failed
 - Unclassified
Classifying C6H11O14-6OOH => C5H9O14-5 + CH2O + OH...
 - Failed
 - Unclassified
Classifying C7H13O24-3OOH => CH3CHCHO + NC3H7CHO + OH...
 - Failed
 - Unclassified
Classifying I3C6D1 + O2CHO = I3C6D1-4 + HO2CHO...
 - Succeeded
Classifying C7H15-4 + O2 = C7H14-3 + HO2...
 - Succeeded
Classi

 - Succeeded
Classifying C5H10-2 + CH3O2 = C5H92-4 + CH3O2H...
 - Succeeded
Classifying C7H14OOH2-3 = C7H14O2-3 + OH...
 - Succeeded
Classifying C6KET25O = CH3CHO + CH2CH2COCH3...
 - Succeeded
Classifying C4H7CHO-4 + CH3 = C4H7CO1-4 + CH4...
 - Succeeded
Classifying IC6KET1-3 = IC6KET1-3O + OH...
 - Succeeded
Classifying C6H5CH3 + O2 = C6H5CH2 + HO2...
 - Succeeded
Classifying C6H11Q14-5 = C6H11O23-6OOH + OH...
 - Succeeded
Classifying C7H131-3 + O2 = C7H121-3 + HO2...
 - Succeeded
Classifying CH3 + C5H81-3 = I3C6D1-5...
 - Succeeded
Classifying C6H102OOH4-6 + O2 = C6H102OOH4-6O2...
 - Succeeded
Classifying IC6KET3-1 = IC6KET3-1O + OH...
 - Succeeded
Classifying C7H13Q14-7 = C5H91-2,5OOH + C2H4...
 - Succeeded
Classifying C7H15-2 + HNO = NC7H16 + NO...
 - Succeeded
Classifying IC6-2 = IC4H8 + C2H5...
 - Succeeded
Classifying NC7H16 = C7H15-1 + H...
 - Succeeded
Classifying C6H101OOH3-4 = C6H10D13 + HO2...
 - Succeeded
Classifying C6H12OOH1-2O2 + CH3 = CH3O + C6H12OOH1-2O...
 - Succeede

 - Succeeded
Classifying XC6D1 + CH3O2 = XC6D1-3 + CH3O2H...
 - Succeeded
Classifying C7H13Q34-5 = C7H13-3D5OOH + HO2...
 - Succeeded
Classifying HO2 + C6H10D24 = C6H102OOH5-4...
 - Succeeded
Classifying C6H12-1 + CH3 = C6H111-4 + CH4...
 - Succeeded
Classifying IC6-3O2 = IC6OOH3-4...
 - Succeeded
Classifying C7H131-3O2 = C7H121OOH3-1...
 - Succeeded
Classifying C6KET12 = C6KET12O + OH...
 - Succeeded
Classifying C6H5CHO + OH = C6H5CO + H2O...
 - Succeeded
Classifying PC4H9CHO + CH3O2 = PC4H8CHO-4 + CH3O2H...
 - Succeeded
Classifying C6H12-1 + O2 = C6H111-1 + HO2...
 - Succeeded
Classifying C7H14-1 + HO2 = C7H131-3 + H2O2...
 - Succeeded
Classifying C7H121OOH6-4 = C7H121O4-6 + OH...
 - Succeeded
Classifying CYC6H7 = C6H6 + H...
 - Succeeded
Classifying C6H101OOH5-6O2 = NC6D1KET56 + OH...
 - Failed
 - Unclassified


In [ ]:
print("Classifying reactions for for NUIG-Pyro")
PYR_RXN_DF = classify_reactions(PYR_RXN_DCT, PYR_SPC_DF)

SUCC_PYR_RXN_PATH = 'reactions/03_reactions-succ_nuig-pyro.csv'
FAIL_PYR_RXN_PATH = 'reactions/03_reactions-fail_nuig-pyro.csv'

SUCC_PYR_RXN_DF = PYR_RXN_DF[PYR_RXN_DF['succeeded']]
FAIL_PYR_RXN_DF = PYR_RXN_DF[~PYR_RXN_DF['succeeded']]

SUCC_PYR_RXN_DF.to_csv(SUCC_PYR_RXN_PATH, quotechar="'")
FAIL_PYR_RXN_DF.to_csv(FAIL_PYR_RXN_PATH, quotechar="'")

In [ ]:
# 4. visualize reactions
def visualize_reactions(rxn_df, spc_df):
    for num, row in rxn_df.iterrows():
        print(f'Row {num}')
        name = row['name']
        print(f'{name}')
        class_ = row['class']
        if class_ is not None:
            print(f' - Class: {class_}')
        rct_names, prd_names, _ = chemkin_io.parser.reaction.get_rxn_name(name)
        rct_smis = [helpers.lookup_value(spc_df, 'smiles', {'name': n}) for n in rct_names]
        prd_smis = [helpers.lookup_value(spc_df, 'smiles', {'name': n}) for n in prd_names]
        rxn_smi = automol.smiles.reaction(rct_smis, prd_smis)
        print(f' - SMILES: {rxn_smi}')
        ts_chi = row['ts_amchi']
        print(f' - TS AMChI: {ts_chi}')
        rct_chis = [helpers.lookup_value(spc_df, 'inchi', {'name': n}) for n in rct_names]
        prd_chis = [helpers.lookup_value(spc_df, 'inchi', {'name': n}) for n in prd_names]
        automol.chi.display_reaction(rct_chis, prd_chis)

In [ ]:
print('Successfully classified NUIG-C6-7 reactions:')
print("WARNING: RDKit fails to properly visualize some radicals")
visualize_reactions(SUCC_C67_RXN_DF, C67_SPC_DF)

In [ ]:
print('Successfully classified NUIG-Pyro reactions:')
print("WARNING: RDKit fails to properly visualize some radicals")
visualize_reactions(SUCC_PYR_RXN_DF, PYR_SPC_DF)

In [ ]:
print('Failed (unclassified) NUIG-C6-7 reactions:')
print("WARNING: RDKit fails to properly visualize some radicals")
visualize_reactions(FAIL_C67_RXN_DF, C67_SPC_DF)

In [ ]:
print('Failed (unclassified) NUIG-Pyro reactions:')
print("WARNING: RDKit fails to properly visualize some radicals")
visualize_reactions(FAIL_PYR_RXN_DF, PYR_SPC_DF)